In [1]:
 # Import libraries
import pandas as pd
import pickle as pk

# import libraries for api
import os
from dotenv import load_dotenv 
import requests
import json

# import libraries for vizualization
import plotly.express as px
import panel as pn
from pprint import pprint

In [3]:
### Open data from a pickle
filename = "Zillow_data"
infile = open(filename,'rb')
new_dict = pk.load(infile)
infile.close()

In [4]:
pprint(new_dict.text)

('{"props":[{"bathrooms":2,"propertyType":"SINGLE_FAMILY","lotAreaValue":1742.4,"address":"318 '
 'W Ruscomb St, Philadelphia, PA '
 '19120","imgSrc":"https:\\/\\/photos.zillowstatic.com\\/fp\\/c528e4507f6a3a9faa6a6397b3b76d10-p_e.jpg","price":150000,"listingDateTime":null,"listingStatus":"FOR_SALE","zpid":"10428068","longitude":-75.13041,"latitude":40.026787,"lotAreaUnit":"sqft","daysOnZillow":-1,"bedrooms":4,"country":"USA","currency":"USD","livingArea":1440,"hasImage":true},{"bathrooms":2,"propertyType":"SINGLE_FAMILY","lotAreaValue":3920.4,"address":"4913 '
 'Castor Ave, Philadelphia, PA '
 '19124","imgSrc":"https:\\/\\/photos.zillowstatic.com\\/fp\\/84229141ec1a1182fc4dde2a43f49ff9-p_e.jpg","price":325000,"listingDateTime":null,"listingStatus":"FOR_SALE","zpid":"10291269","longitude":-75.09111,"latitude":40.024803,"lotAreaUnit":"sqft","daysOnZillow":-1,"bedrooms":4,"country":"USA","currency":"USD","livingArea":1920,"hasImage":true},{"bathrooms":4,"propertyType":"SINGLE_FAMILY","lo